In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

This notebook shows how supply sensitive routing works using a randomly generated graph. The core concepts are outlined in the associated paper. In this notebook, a random graph containing places and stations is generated. First a generic example is given where routing is done purely to minimize travel time. Second, a case including vehicle and station parameters is generated and run.

In [33]:
'''
Creating a random graph

graph contains n places and m charging stations - the random graph is completely connected
'''
deep_reload(src)

# Parameters for graph creation
seed = 125396749 # RNG seed [-]
n = 15 # Number of places [-]
m = 85 # Number of stations [-]
speeds = [105 / 3.6] # Set of speeds to be assigned to links [m/s]
chargers = list(range(10)) # Set of numbers of chargers to be assigned to stations [-]
scale = (1e6, 1e6) # Scale of the area on which the graph is generated ([m], [m])

# Setting the graph as directed
kw = {
    'graph': {
        'directed': True,
    },
}

# Creating the graph
graph = src.rng.random_completely_connected_graph(
    n, m, speeds = speeds, scale = scale, seed = seed, chargers = chargers, **kw
)

# # Marking the edge types by source and target type
# graph = src.routing.edge_types(graph)

In [40]:
adjacency = nx.to_numpy_array(graph, weight = 'time')
adjacency[adjacency > 2 * 3600] = np.inf

In [41]:
adjacency

array([[   0.        ,           inf,           inf, ..., 6295.49467068,
                  inf,           inf],
       [          inf,    0.        , 4682.7811226 , ...,           inf,
                  inf,           inf],
       [          inf, 4682.7811226 ,    0.        , ...,           inf,
                  inf,           inf],
       ...,
       [6295.49467068,           inf,           inf, ...,    0.        ,
                  inf,           inf],
       [          inf,           inf,           inf, ...,           inf,
           0.        , 4071.12196209],
       [          inf,           inf,           inf, ...,           inf,
        4071.12196209,    0.        ]])

In [47]:
from scipy.sparse.csgraph import floyd_warshall

_, previous = floyd_warshall(adjacency, return_predecessors = True)

In [48]:
previous

array([[-9999,    17,    91, ...,     0,    60,    25],
       [   94, -9999,     1, ...,    62,    59,    18],
       [   94,     2, -9999, ...,    62,    95,    48],
       ...,
       [   97,    65,    17, ..., -9999,    99,    84],
       [   94,    17,    91, ...,    84, -9999,    98],
       [   94,    17,    91, ...,    84,    99, -9999]], dtype=int32)

In [51]:
u = 0
v = 55

path = [v]

k = 0
while u != v:
    
    k += 1
    if k > 100:
        break
    
    v = previous[u][v]
    path.append(v)

path   

[55, 31, 95, 99, 25, 94, 0]

In [52]:
graph.is_directed()

True

In [59]:
'''
Running with Bellman with default objective

This cell uses a simple objective function which minimizes travel time but limits
the travel time of edge which can be traversed forcing indirect routes in some cases.
This is a fairly limiting cost function which, more or less mimics the NetworkX
implementation.
'''
deep_reload(src)

# Selecting the origin - this must be an iterable. Multiple origins can be selected.
origins = [k for k in graph.nodes if 'place' in k]
pivots = [k for k in graph.nodes if 'place' not in k]
# origins = ['place_0']

# Defining the cost function
objective = src.floyd_warshall.Objective(field = 'time', edge_limit = 3600 * 2)

t0 = time.time()

# Running the optimization
costs, values, paths = src.floyd_warshall.floyd_warshall(
    graph, origins,
    pivots = pivots,
    objective = objective,
    return_paths = True,
)

print(f'Executed in {time.time() - t0:.4f} seconds')

Executed in 0.8562 seconds


In [62]:
costs['place_1']

{'place_0': 32973.00116660798,
 'place_1': 0.0,
 'place_2': 4682.781122601585,
 'place_3': 33600.24587843644,
 'place_4': 27843.99826641951,
 'place_5': 19185.581178251567,
 'place_6': 31816.150408820453,
 'place_7': 17074.77383852056,
 'place_8': 32319.29213570632,
 'place_9': 34161.58203722548,
 'place_10': 15808.378506572184,
 'place_11': 11407.949624792023,
 'place_12': 22789.68032854288,
 'place_13': 15457.268914873042,
 'place_14': 32522.701499528095,
 'station_0': 25728.477930775072,
 'station_1': 22814.099129779446,
 'station_2': 5243.556605726273,
 'station_3': 11778.093297700783,
 'station_4': 5099.977314052751,
 'station_5': 20711.40161184728,
 'station_6': 29778.138931252724,
 'station_7': 13493.36601860723,
 'station_8': 16196.82621885345,
 'station_9': 31596.59013230054,
 'station_10': 22928.27693932585,
 'station_11': 21336.452331920278,
 'station_12': 21816.19345287699,
 'station_13': 22160.049005360946,
 'station_14': 30591.585146385332,
 'station_15': 20026.2150560115

In [61]:
previous = paths

u = 'place_0'
v = 'place_1'

path = [v]

k = 0
while u != v:
    
    k += 1
    if k > 100:
        break
    
    v = previous[u][v]
    path.append(v)

path   

['place_1',
 'station_2',
 'station_3',
 'station_84',
 'station_10',
 'station_79',
 'place_0']